<span type="title">Java Stand Tag Library</span> | <span type="update">2018-09-24</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 JSTL。JSTL 是 Java EE 的一个标准标签库，用于提供对于 EL 的逻辑支持。在上一章我们介绍了 Stand Action 以及 EL，这两个标准可以用来创建 Beans 和使用 Beans，以及使用映射，进行一些简单的算术和逻辑运算。但是，这对于完整的提供View层还不够，我们还需要像是条件判断、多项选择、错误处理等等的更像是一门语言的标准。这就是 JSTL。在 JSTL 中使用 EL 表达式能够完成更加丰富的操作。</p><p class="card-text">本文主要介绍Core前缀的 c:out, c:forEach, c:if, c:choose, c:when, c:otherwise, c:set, c:remove, c:import, c:url, c:catch。此外还有用于 sql 和 xml 处理的特定前缀和标签。此外，本文还介绍了TLD以及自定义标签处理器来创建自己的标签的方法。这在一定程度上可以弥补JSTL的不足，虽然大多数情况下JSTL即可很好的完成工作。</p></span>

# c:out

c:out 标签用来输出打印值，并且提供一些额外的操作。

```jsp
<h1>Welcome, ${pageScope.person.name}</h1>

<p>${pageScope.person.name}, You are <c:out value="${pageScope.person.age}" default="here" escapeXml="true" /> now.</p>
```

不难看出，C:OUT更加复杂，相比较直接使用EL而言。但是，这样有额外的好处，比如，我们可以使用 default 参数填充默认值，使用 excapeXml 参数来将XML解析为HTML，避免双引号、单引号、尖括号和and号的转义问题（默认开启）。

更为有用的是，c:out 避免了用户输入指令的攻击，防止程序产生预期以外的错误。因此，对于任何可能来自用户输入的地方，都需要强制使用 c:out，而不能直接使用 EL 打印值。c:out 的必选参数是 value，其可以接受运行时参数传入（可以计算得到一个值）。

# c:forEach

这个标签是c库唯一用来遍历容器元素的标签，其可以接受一个iterable的对象，放置在 items 参数中。当标签被执行时，其会在标签作用域创建一个 var 定义好的本地变量，使用指定的 step、start和 end属性来对于 items 的内容进行遍历，每次遍历的值都放在 var 中。注意，此标签还接受一个 varStatus 的指定，这个标签作用域内的本地变量可以在每次遍历时提供当前的指针位置、指针指向的对象。

```jsp
<c:forEach var="word" items="${pageScope.words}" step="1" begin="0" end="3" varStatus="status">
    <p>${status.count} - ${word}</p>
</c:forEach>
```
forEach 可以嵌套，其中被嵌套的标签在外部标签作用域内，可以访问作用域范围的本地变量 group。对下面的例子而言，school是一个列表的列表，group是school的列表，在子循环中，我们用group作为需要被遍历的对象，创建更深的本地变量 item 来取出 group 中的 Person 对象，使用 EL 表达式进行值的获取。

```jsp
<table><c:forEach varStatus="status" var="group" items="${pageScope.school}">
    <tr>
        <c:forEach varStatus="p_status" var="item" items="${group}">
            <td>${item.name}</td>
            <td>${item.age}</td>
            <td>${item.address}</td>
        </c:forEach>
    </tr>
</c:forEach></table>
```

需要注意，所有的标签作用域内的变量都可以使用 ${xxx} 来获取。这是因为，如果不加Scope前缀，默认会使用当前最低等级的作用域，对于 page 而言，就是 pageScope，对于标签而言，则是标签的作用域。出了标签，这个变量将变成 page 作用域下的 Attribute 属性，比如通过 setAttribute 来设置才能访问。

# c:if

这是一个残缺的 IF 条件逻辑，因为没有 else 部分。if 标签接受 test 作为判断逻辑，这个 test 必须可以计算得到一个 bool 值。此外，还可以有 var 属性，这个属性的值将被用作设置为  scope 属性指定的作用域的属性。如果不指定 scope 属性的话，默认是 page 作用域。

**EL表达式和标签属性的作用域推断逻辑不同：** 当标签的属性不设置，则默认为 page，固定死的。当EL表达式不设置作用域，那么从下往上开始查找，找到为止。

```jsp
<c:if test="${param.word eq 'hello'}" var="keys">
    You think the word is hello, you are right!<br/>
</c:if>

<c:if test="${keys eq true}" var="result" scope="page">
    I got key value from pageScope, it is hi.<br/>
</c:if>
${result}
```

# c:choose

为了进行多项选择，使用 choose 和 when。

```jsp
<c:choose>
    <c:when test="${param.word eq 'hi'}">
        You input hi, it is cool <br/>
    </c:when>
    <c:when test="${param.word eq 'hello'}">
        You input hello, it is not bad.
    </c:when>
    <c:otherwise>
        You input ${param.word}. It is just so so.
    </c:otherwise>
</c:choose>
```

注意，这里的 choose 和 switch 不同，如果满足其中一项，则直接全部返回，后面不再执行。你可以理解 choose 和 when 是高级版本的 if - else。when 需要接受一个 test 的布尔值输入，如果符合则进行其体中的语句，否则跳过。otherwise 表示如果都不符合，则执行，类似于最后的 else。

# c:set

除了使用脚本，在之前只能通过 useBean 来构建 JAVA 对象。但是，那样的话，传入 Beans 属性有类型的限制，不能是非基本类型或者String的。而 c:set 就没有这种限制，并且提供了更加强大的功能。c:set 分为两类，其一是向各个作用域传递Bean/Map等属性的set，其二是向某个特定的Beans/Map设置属性/映射的set。

## 设置 scope 的属性

```jsp
<c:set var="identity" scope="page" value="Marvin"/>
<c:set var="person" value="null" />
<c:set var="person">
    Corkine Ma, Lucy ${param.word}
</c:set>
```

**可以有体：** 体中可以包含更为复杂的，需要经过计算得到的内容。

**利用value或者体创建了本地变量var：** 在这种情况下，首先创建一个包含 value 值或者 体 的值的本地变量 var。

**利用var向作用域覆盖写值：** 标签将 var 的值赋值给指定 scope 的同名属性 identity, 覆盖写入。此外，注意，不写scope，默认为page。

**如果是null直接删除作用域属性：** 需要注意，如果value的值为null，那么即便是原来有一个同名变量，那个同名变量也会被删除。可以假想逻辑是这样的，本地变量强制setAttribute，然后getAttritube进行检查，如果是null，那么直接删除这个属性。

## 设置 Beans/映射的属性

```jsp
<c:set target="${lili}" property="name" value="Corkine" />
<c:set target="${lili}" property="address">Central China Normal University</c:set>
```

在这种模式下，需要有 target 来标识一个 Beans 或者映射。需要注意，这里不能是String字面量，这里不是指的 Beans 或者 映射 的 id 属性，需要这个 Beans 或者 映射本身。property 用来标记需要向哪个字段传值。value 表明传递的值时什么，如果不指定，则使用体中的字符串传入。此外，尤其需要注意，如果 target 的类型不是 Map 或者 Beans，而是 List 或者 Array 也会出错。此外，target 不能为 null，否则会出错。此外，property 不能找不到，否则也会出错。可以说，**我们必须在使用 set 给 Beans/Map 设置的时候，确保有一个非空对象的存在属性。**

## 区分 scope 属性和 Beans 属性设置

和设置 scope 的共同点在于：都必须提供一个输入值，通过 value 或者 体，区别在于，设置 scope 需要提供一个本地变量var 以及 scope，会向 scope 同名属性进行设置。而设置Beans/映射则需要指定非null的 target 对象以及 property，并且直接进行设置。

对于 scope 设置，我们更关心 value 为 null 的问题，因为这样可能导致 scope 属性被删除。对于 Beans 设置，我们更关心 target 的问题。因为这个 target 不能为 null，对类型有要求，对 property 的存在也有要求，否则就会出错。

# c:remove

我们可以用 c:set value="null" var="targetAttribute" scope="page" 来删除 page.targetAttribute 这个属性，但是，依旧存在更加正常的删除方式：c:remove。

```jsp
${lili}
<c:remove var="lili" scope="page"/>
```
remove 有一些尤其需要注意的点：var 必须为String字面量，不能经过计算。scope如果不写，则从所有的作用域中全部删除此值。

# c:import

这个标签和 jsp:include 以及 include 指令均可用来在页面中包含页面。此外，使用 taglib 也可以，在后面会讲到。

对于 c:import 而言，其动态运行时添加，和 jsp:include 类似。但是和后者参数不同，include 指令使用 file，jsp：include 使用 page，而 c:import 使用 url 参数。

相比较 jsp:include 标准董总，c:import 标签的 url 属性可以包含 WEB 以外的资源。此外，可以正常的在体中添加 param 属性。在片段中使用 ${param.name} 来获取 value。

```jsp
<c:import url="/header.html">
    <c:param name="date" value="${m:time()}"/>
</c:import>
```

# c:url

c:url 是用来输出 url 的标记。它可以动态跟踪客户端，如果不允许使用 cookie，则会使用 url 编码来确保回话跟踪。c:url 可以设置字符串查询参数，使用 c:param 标签内嵌即可。比如: `learn_jstl.jsp?name=Corkine+Ma&id=232323`

```jsp
<c:url value="learn_jstl.jsp">
    <c:param name="name" value="Corkine Ma"/>
    <c:param name="id" value="232323"/>
</c:url>
```

# c:catch 和 errorPage

## EE 错误页面处理

错误页面处理可以使用 DD 来根据 错误响应代码 和 Java Exception 类型来指定对应的 JSP 页面。也可以使用 page 标签为当前页面指定一个具体的 errorPage 页面。

**需要注意的是，这两种方案的路径，都必须设置为绝对路径，即相对于APPLICATION的路径**。错误页面的声明只能有1次，并且必须放置在错误发生之前（也就是页面最顶端，如果采用page指定设置的话）。

```jsp
<%@ page errorPage="/errorPage.jsp" %>
```

```xml
<error-page>
    <exception-type>java.lang.Throwable</exception-type>
    <location>/errorPage.jsp</location>
</error-page>
<error-page>
    <error-code>404</error-code>
    <location>/404.jsp</location>
</error-page>
```

Throwable 默认捕获所有类型的错误。对于错误页面而言，其必须声明page指令 `<%@ page isErrorPage="true" %>`，此外，可以通过 `${pageContext.exception}` 获取错误信息。

## c:catch 捕获错误

```jstl
<c:catch var="Myexception">
    dosomething wrong
</c:catch>
<c:if test="${myException ne null}">
    something wrong: ${myException.message}
</c:if>
```

这个语法很简单，catch 语法块之间的东西就是我们需要处理的代码。如果发生错误，仅仅将错误写入 var 指定的 page 的 myException 属性中，然后跳出代码块，执行JSP页面的其余部分。

需要注意，发生错误的时候，只有显示错误的页面才会有 exception 对象，而发生错误的页面则没有，因此不能直接使用 `${pageContext.exception}` 在这个正常页面获取异常。这个 var 的 exception 也不是标记作用域，而是 page 的作用域，在catch标签之外，可以直接获得。

# 定制标签处理器

## tld 文件和 tag 使用

在上一章，我们介绍过通过 tld 文件来使用 taglib，以提供 EL 通过 taglib 来调用 Java 类和方法的能力。调用的写法是：`${head:function()}`。tld 文件除了提供 function 调用之外，还可以提供 tag 的调用和处理。与之相对应的，我们需要一个 Java 类来处理 tag，这就是标签处理器。这是两种不同的思路，以 HTML 为先还是以 Java 为先。

一个 tld 文件如下：

```xml
<uri>RollMe</uri>

<tag>
    <description>give you some advice.</description>
    <name>advice</name>
    <tag-class>com.mazhangjing.model.Advicer</tag-class>
    <body-content>empty</body-content>
    <attribute>
        <name>user</name>
        <required>true</required>
        <rtexprvalue>true</rtexprvalue>
    </attribute>
</tag>


<function>
    <name>roll</name>
    <function-class>com.mazhangjing.model.DiceRoller</function-class>
    <function-signature>int rollIt()</function-signature>
</function>
```

其中对于 tag 而言，最重要的是 tag 的 name 标签、接受处理的 tag-class 标签、允许包含内容类型的 body-content 标签、以及各个属性字段的 attribute 标签。

对于 attribute 而言，需要有 name 标签、是否强制的 required 标签、填写内容是否允许计算的 rtexprvalue 标签（默认为true）。在需要调用的 JSP 页面的用法如下：

```jsp
<%@ taglib prefix="m" uri="RollMe" %>
<m:advice user="${param.user}"/>
```

需要注意，如果 body-content 为 empty，也就是不允许有体，通过 jsp:attribute 在体中来设置属性也是可以的。body-content 的其余选择为： empty、scriptless（不能有脚本表达式、声明和Scriptlet，只含有EL或者模板文本）、tagdependent（纯文本）、jsp（任何可以放在JSP中的东西）。

## 标签处理器

对于标签处理器，最简单的方式是继承 SimpleTagSupport。

```java
public class Advicer extends SimpleTagSupport {
    private String user;
    @Override
    public void doTag() throws JspException, IOException {
        getJspContext().getOut().write("Hello, " + user);
        getJspContext().getOut().write(getAdvice());
    }

    public void setUser(String user) {
        this.user = user; //容器自动调用这个方法传递参数user。
    }

    String getAdvice() { return "Live long";}
}
```

其中在 doTag 中处理标签事务。设置类实例用于标签属性的值的获取。使用 getJspContext 来输出内容。
   
## tld 文件的路径

tld 中的 uri 只要是一个独特字串就可以, 2.0 以后的标准容器自动检测 tld 文件，不需要在 DD 中设置。当然，JSP 2.0之前，包括现在，依然可以设置，设置后会自动通过设置来寻找。

```xml
<jsp-config>
    <taglib>
        <taglib-uri>random</taglib-uri>
        <taglib-location>/WEB-INF/cmSite.tld</taglib-location>
    </taglib>
</jsp-config>
```

如果不进行 taglib 的配置，那么容器会在以下位置寻找 tld 文件：

- WBE-INF
- WBE-INF 的子目录
- WEB-INF/lib的Jar的META-INF
- WEB-INF/lib的Jar的META-INF的子目录